# Quantization modules

In [1]:
import torch
from torch.autograd.function import InplaceFunction, Function
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.autograd import Variable
import math
import numpy as np
import torch.nn.init as init
import torchvision
from tqdm import tqdm
import json
from pathlib import Path

In [3]:
def quantize_model(model, quantize = False, bits = 8, qmode = "dynamic"):
    if quantize:
        print("Quantize mode on")
        for layer in model.modules():
            try:
                mode = layer.mode()
                if mode == False:
                    layer.change_mod(True, bits, qmode)
            except:
                continue
    else:
        print("Quantize mode off")
        for layer in model.modules():
            try:
                mode = layer.mode()
                if mode == True:
                    layer.change_mod(False, 0)
            except:
                continue
    return model

In [4]:
def qsin_activation_mode(model):
    print("QSIN activation mode on")
    for layer in model.modules():
        try:
            mode = layer.mode()
            layer.qsinmode()
        except:
            continue
    return model

In [5]:
class MyRound(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        return torch.round(input)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output

In [6]:
def Quantize_tensor(input_tensor, max_abs_val = None, num_bits = 8):
    my_round = MyRound.apply
    qmin = -1.0 * (2**num_bits) / 2
    qmax = -qmin - 1
    scale = max_abs_val / ((qmax - qmin) / 2)
    input_tensor = torch.div(input_tensor, scale)
    input_tensor = my_round((input_tensor))
    input_tensor = torch.clamp(input_tensor, qmin, qmax)
    return torch.mul(input_tensor, scale)

In [7]:
class Quant(nn.Module):
    def __init__(self, num_bits=8, mode = "dynamic", static_count = 30):
        super(Quant, self).__init__()
        self.num_bits = num_bits
        self.mode = mode
        self.static_count = static_count
        self.static_cur = 0
        self.stat_values = []
        self.max_abs = 0 
        if mode != "dynamic":
            self.max_abs_tr = nn.Parameter(torch.zeros(0), requires_grad=True) # IMPORTANT
        
    def forward(self, input):
        if self.mode == "dynamic":
            self.max_abs = torch.max(torch.abs(input.detach()))
            return Quantize_tensor(input, self.max_abs, self.num_bits)
        
        elif self.mode == "static":
            if self.static_cur > self.static_count:
                return Quantize_tensor(input, self.max_abs_tr, self.num_bits)
            elif self.static_cur == self.static_count:
                self.max_abs = np.mean(self.stat_values)
                self.max_abs_tr.data = torch.tensor(self.max_abs, dtype=torch.float).to(self.max_abs_tr.device)
                self.static_cur += 1
                return Quantize_tensor(input, self.max_abs_tr, self.num_bits)
            else:
                self.static_cur += 1
                self.stat_values.append(np.max(np.absolute(input.cpu().detach().numpy())))
                return input

In [8]:
def QSin(x, num_bits = 8): 
    pi = torch.tensor(np.pi)
    qmin = -1.0 * (2**num_bits) / 2
    qmax = -qmin - 1
    result = torch.sum(torch.square(torch.sin(torch.mul(pi, x[torch.logical_and(x >= qmin, x <= qmax)]))))
    result = result + torch.sum(torch.mul(torch.square(pi), torch.square((x[x < qmin] - qmin))))
    result = result + torch.sum(torch.mul(torch.square(pi), torch.square((x[x > qmax] - qmax))))
    return result

In [9]:
class Linear(nn.Linear):
    def __init__(self, in_features: int, out_features: int, bias: bool = True, quantization: bool = False, q_bits: int = 8, qsin_activation = False):
        super(Linear, self).__init__(in_features, out_features, bias)

        self.quantize = True if quantization else False
        self.QsinA = True if qsin_activation else False

        if self.quantize:
            self.bits = q_bits
            self.Quantize_weights = Quant(self.bits)
            self.Quantize_input = Quant(self.bits)
        else:
            self.bits = 'FP'
            
        if self.QsinA:
            self.qsin_loss_A = 0

    def init(self, input):
        self.inputW = input.shape
        
    def change_mod(self, value, bits = 8, mode = "dynamic"):
        self.quantize = value
        self.bits = bits
        self.Quantize_weights = Quant(bits, mode)
        self.Quantize_input = Quant(bits, mode)
        
    def qsinmode(self):
        self.QsinA = True
        self.qsin_loss_A = 0
        
    def mode(self):
        return self.quantize  

    def forward(self, input):
            
        if self.quantize:
            qinput = self.Quantize_input(input)
            qweight = self.Quantize_weights(self.weight)
            
            #count qsin loss on activation
            if self.QsinA:
                self.qsin_loss_A = 0
                qmin = -1.0 * (2**self.bits) / 2
                qmax = -qmin - 1
                scale = self.Quantize_input.max_abs_tr / ((qmax - qmin) / 2)
                sq_scale = torch.square(scale)
                self.qsin_loss_A = torch.mul(sq_scale, QSin(torch.div(input, scale), self.bits))
            
            return nn.functional.linear(qinput, qweight, self.bias)
        else:
            return nn.functional.linear(input, self.weight, self.bias)

In [10]:
class Embedding(nn.Embedding):
    def __init__(self, num_embeddings, embedding_dim, padding_idx=None, max_norm=None,
                 norm_type=2.0, scale_grad_by_freq=False, sparse=False,
                 quantization: bool = False, q_bits: int = 8):
        super(Embedding, self).__init__(num_embeddings, embedding_dim, padding_idx)

        self.quantize = True if quantization else False

        if self.quantize:
            self.bits = q_bits
            self.Quantize_weights = Quant(self.bits)
            self.Quantize_input = Quant(self.bits)
        else:
            self.bits = 'FP'

    def init(self, input):
        self.inputW = input.shape
        
    def change_mod(self, value, bits = 8, mode = "dynamic"):
        self.quantize = value
        self.bits = bits
        self.Quantize_weights = Quant(bits, mode)
        self.Quantize_input = Quant(bits, mode)
        
    def mode(self):
        return self.quantize  

    def forward(self, input):
            
        if self.quantize:
            qweight = self.Quantize_weights(self.weight)
        
            return nn.functional.embedding(input, qweight, self.padding_idx, self.max_norm,
                 self.norm_type, self.scale_grad_by_freq, self.sparse)
        else:
            return nn.functional.embedding(input, self.weight, self.padding_idx, self.max_norm,
                 self.norm_type, self.scale_grad_by_freq, self.sparse)
        

In [11]:
def Qsin_W(model, bits = 8):
    qmin = -1.0 * (2**bits) / 2
    qmax = -qmin - 1
    loss = 0
    for layer in model.modules():
        try:
            scale = layer.Quantize_weights.max_abs_tr / ((qmax - qmin) / 2)
            sq_scale = torch.square(scale)
            QSin_w = QSin(torch.div(layer.weight, scale), bits)
            loss = loss + torch.mul(sq_scale, QSin_w)
        except:
            continue
    return loss

In [12]:
def Qsin_A(model):
    loss = 0
    for layer in model.modules():
        try:
            loss = loss + layer.qsin_loss_A
        except:
            continue
    return loss

In [13]:
def get_custom_Linear(in_features, out_features, bias, weight):
    linear = Linear(in_features, out_features)
    linear.bias = bias
    linear.weight = weight
    return linear

def get_custom_Embeding(num_embeddings, embedding_dim, padding_idx, weight):
    embedding = Embedding(num_embeddings, embedding_dim, padding_idx)
    embedding.weight = weight
    return embedding

def change_layers(model):
    for name, layer in model.named_children():
        if isinstance(layer, nn.Linear):
            setattr(model, name, get_custom_Linear(
                                                layer.in_features,
                                                layer.out_features,
                                                layer.bias,
                                                layer.weight
            ))
            
        if isinstance(layer, nn.Embedding):
            setattr(model, name, get_custom_Embeding(
                                                layer.num_embeddings,
                                                layer.embedding_dim,
                                                layer.padding_idx,
                                                layer.weight
            ))
        change_layers(getattr(model, name))

# glue metric 

In [14]:
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import f1_score, matthews_corrcoef

import datasets


_CITATION = """\
@inproceedings{wang2019glue,
  title={{GLUE}: A Multi-Task Benchmark and Analysis Platform for Natural Language Understanding},
  author={Wang, Alex and Singh, Amanpreet and Michael, Julian and Hill, Felix and Levy, Omer and Bowman, Samuel R.},
  note={In the Proceedings of ICLR.},
  year={2019}
}
"""

_DESCRIPTION = """\
GLUE, the General Language Understanding Evaluation benchmark
(https://gluebenchmark.com/) is a collection of resources for training,
evaluating, and analyzing natural language understanding systems.
"""

_KWARGS_DESCRIPTION = """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:
    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(results)
    {'accuracy': 1.0}
    >>> glue_metric = datasets.load_metric('glue', 'mrpc')  # 'mrpc' or 'qqp'
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(results)
    {'accuracy': 1.0, 'f1': 1.0}
    >>> glue_metric = datasets.load_metric('glue', 'stsb')
    >>> references = [0., 1., 2., 3., 4., 5.]
    >>> predictions = [0., 1., 2., 3., 4., 5.]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print({"pearson": round(results["pearson"], 2), "spearmanr": round(results["spearmanr"], 2)})
    {'pearson': 1.0, 'spearmanr': 1.0}
    >>> glue_metric = datasets.load_metric('glue', 'cola')
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(results)
    {'matthews_correlation': 1.0}
"""


def simple_accuracy(preds, labels):
    return (preds == labels).mean()


def acc_and_f1(preds, labels):
    acc = simple_accuracy(preds, labels)
    f1 = f1_score(y_true=labels, y_pred=preds)
    return {
        "accuracy": acc,
        "f1": f1,
    }


def pearson_and_spearman(preds, labels):
    pearson_corr = pearsonr(preds, labels)[0]
    spearman_corr = spearmanr(preds, labels)[0]
    return {
        "pearson": pearson_corr,
        "spearmanr": spearman_corr,
    }


@datasets.utils.file_utils.add_start_docstrings(_DESCRIPTION, _KWARGS_DESCRIPTION)
class Glue(datasets.Metric):
    def _info(self):
        if self.config_name not in [
            "sst2",
            "mnli",
            "mnli_mismatched",
            "mnli_matched",
            "cola",
            "stsb",
            "mrpc",
            "qqp",
            "qnli",
            "rte",
            "wnli",
            "hans",
        ]:
            raise KeyError(
                "You should supply a configuration name selected in "
                '["sst2", "mnli", "mnli_mismatched", "mnli_matched", '
                '"cola", "stsb", "mrpc", "qqp", "qnli", "rte", "wnli", "hans"]'
            )
        return datasets.MetricInfo(
            description=_DESCRIPTION,
            citation=_CITATION,
            inputs_description=_KWARGS_DESCRIPTION,
            features=datasets.Features(
                {
                    "predictions": datasets.Value("int64" if self.config_name != "stsb" else "float32"),
                    "references": datasets.Value("int64" if self.config_name != "stsb" else "float32"),
                }
            ),
            codebase_urls=[],
            reference_urls=[],
            format="numpy",
        )

    def _compute(self, predictions, references):
        if self.config_name == "cola":
            return {"matthews_correlation": matthews_corrcoef(references, predictions)}
        elif self.config_name == "stsb":
            return pearson_and_spearman(predictions, references)
        elif self.config_name in ["mrpc", "qqp"]:
            return acc_and_f1(predictions, references)
        elif self.config_name in ["sst2", "mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]:
            return {"accuracy": simple_accuracy(predictions, references)}
        else:
            raise KeyError(
                "You should supply a configuration name selected in "
                '["sst2", "mnli", "mnli_mismatched", "mnli_matched", '
                '"cola", "stsb", "mrpc", "qqp", "qnli", "rte", "wnli", "hans"]'
            )

# BERT

In [15]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('./models/model-bert-base/', local_files_only=True)

In [15]:
from datasets import load_dataset, load_metric
from transformers import DistilBertTokenizer
    
batch_size = 32

tokenizer = DistilBertTokenizer.from_pretrained('./models/tokenizer-bert-base/', local_files_only=True)

In [16]:
from datasets import load_from_disk
encoded_dataset = load_from_disk('cur_glue_data')

In [17]:
from transformers import TrainingArguments, Trainer
import numpy as np
task = 'stsb'
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
metric = metric = Glue(task)


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)


In [1]:
change_layers(model)

In [19]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"

In [23]:
args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Pearson,Spearmanr,Runtime,Samples Per Second
1,No log,0.569310,0.877687,0.880065,2.263100,662.815000
2,0.863400,0.481069,0.891091,0.887412,2.183900,686.856000
3,0.320900,0.483172,0.893608,0.890437,2.196400,682.950000


TrainOutput(global_step=1080, training_loss=0.567343967932242, metrics={'train_runtime': 154.8641, 'train_samples_per_second': 6.974, 'total_flos': 1393513314445116.0, 'epoch': 3.0})

In [24]:
torch.cuda.empty_cache()

In [22]:
#from transformers import AutoModelForSequenceClassification
#model = AutoModelForSequenceClassification.from_pretrained('./test-glue/checkpoint-720/',
#                                                           local_files_only=True)

In [25]:
change_layers(model)

In [26]:
trainer.evaluate()

{'eval_loss': 0.4810692071914673,
 'eval_pearson': 0.8910907832498873,
 'eval_spearmanr': 0.8874122931185459,
 'eval_runtime': 2.1953,
 'eval_samples_per_second': 683.276}

In [30]:
model = quantize_model(model, quantize=False, bits = 8)
model = quantize_model(model, quantize=True, bits = 8)

Quantize mode off
Quantize mode on


In [31]:
Eval_DQ = Trainer(
    model,
    args,
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [32]:
Eval_DQ.evaluate()

{'eval_loss': 1.370012640953064,
 'eval_pearson': 0.8496805226953267,
 'eval_spearmanr': 0.8628488003329071,
 'eval_runtime': 3.4085,
 'eval_samples_per_second': 440.072}

In [39]:
model = quantize_model(model, quantize=False, bits = 8)
model = quantize_model(model, quantize=True, bits = 8, qmode = "static")
encoded_dataset_static = load_from_disk('cur_glue_data_st')

Quantize mode off
Quantize mode on


In [40]:
train_enc = encoded_dataset_static['train']

In [41]:
from torch.utils.data import DataLoader
train_enc.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
train_loader = torch.utils.data.DataLoader(train_enc, batch_size=8, shuffle = True)

In [42]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.eval()
i = 0
for batch in tqdm(train_loader):
    i += 1
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    if i == 32:
        break

  4%|▍         | 31/719 [00:24<09:04,  1.26it/s]


In [43]:
args = TrainingArguments(
    "test-glue",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    metric_for_best_model=metric_name,
)

Eval_SQ = Trainer(
    model,
    args,
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [44]:
Eval_SQ.evaluate()

{'eval_loss': 1.4507509469985962,
 'eval_pearson': 0.8559594621511081,
 'eval_spearmanr': 0.8663923035421431,
 'eval_runtime': 2.7595,
 'eval_samples_per_second': 543.579}

# QSIN

In [45]:
qsin_activation_mode(model)
print()

QSIN activation mode on



In [46]:
torch.cuda.empty_cache()

In [47]:
import torch
from transformers import Trainer

class QSinTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        Qsin_W_loss = Qsin_W(model, 8)
        Qsin_A_loss = Qsin_A(model)
        L = outputs[0]
        lambda_w = 10 ** (np.round(np.log10(Qsin_W_loss.cuda().tolist()) - np.log10(L.cuda().tolist())))
        lambda_a = 10 ** (np.round(np.log10(Qsin_A_loss.cuda().tolist()) - np.log10(L.cuda().tolist()))+1)
        loss = L + Qsin_W_loss / lambda_w + Qsin_A_loss / lambda_a
        return (loss, outputs) if return_outputs else loss

In [48]:
args = TrainingArguments(
    "qsin_train_tmp",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    #eval_steps=10,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

trainer_QSin = QSinTrainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [49]:
trainer_QSin.train()

Epoch,Training Loss,Validation Loss,Pearson,Spearmanr,Runtime,Samples Per Second
1,No log,1.298086,0.890405,0.886263,8.809100,170.279000
2,No log,1.176635,0.893213,0.889401,8.868300,169.142000
3,0.491100,1.135844,0.895877,0.892030,8.877200,168.972000


Some weights of the model checkpoint at qsin_train_tmp/checkpoint-540 were not used when initializing BertForSequenceClassification: ['bert.embeddings.word_embeddings.Quantize_weights.max_abs_tr', 'bert.embeddings.word_embeddings.Quantize_input.max_abs_tr', 'bert.embeddings.position_embeddings.Quantize_weights.max_abs_tr', 'bert.embeddings.position_embeddings.Quantize_input.max_abs_tr', 'bert.embeddings.token_type_embeddings.Quantize_weights.max_abs_tr', 'bert.embeddings.token_type_embeddings.Quantize_input.max_abs_tr', 'bert.encoder.layer.0.attention.self.query.Quantize_weights.max_abs_tr', 'bert.encoder.layer.0.attention.self.query.Quantize_input.max_abs_tr', 'bert.encoder.layer.0.attention.self.key.Quantize_weights.max_abs_tr', 'bert.encoder.layer.0.attention.self.key.Quantize_input.max_abs_tr', 'bert.encoder.layer.0.attention.self.value.Quantize_weights.max_abs_tr', 'bert.encoder.layer.0.attention.self.value.Quantize_input.max_abs_tr', 'bert.encoder.layer.0.attention.output.dense.Q

TrainOutput(global_step=540, training_loss=0.47881572864673755, metrics={'train_runtime': 746.3863, 'train_samples_per_second': 0.723, 'total_flos': 1415414277021360.0, 'epoch': 3.0})

# QAT

In [27]:
args = TrainingArguments(
    "qat_train_tmp",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    #eval_steps=10,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)


trainer_QAT = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [28]:
torch.cuda.empty_cache()

In [29]:
trainer_QAT.train()

Epoch,Training Loss,Validation Loss,Pearson,Spearmanr,Runtime,Samples Per Second
1,No log,0.620719,0.885621,0.883652,2.119300,707.776000
2,0.283100,0.449922,0.896620,0.891995,2.121500,707.044000
3,0.158900,0.443404,0.898980,0.894355,2.121900,706.918000


TrainOutput(global_step=1080, training_loss=0.21297242994661683, metrics={'train_runtime': 153.142, 'train_samples_per_second': 7.052, 'total_flos': 1393513314445116.0, 'epoch': 3.0})

# QSIN STEPS

In [ ]:
qsin_activation_mode(model)
print()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import torch
from transformers import Trainer

class QSinTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        Qsin_W_loss = Qsin_W(model, 8)
        Qsin_A_loss = Qsin_A(model)
        L = outputs[0]
        lambda_w = 10 ** (np.round(np.log10(Qsin_W_loss.cuda().tolist()) - np.log10(L.cuda().tolist())))
        lambda_a = 10 ** (np.round(np.log10(Qsin_A_loss.cuda().tolist()) - np.log10(L.cuda().tolist()))+1)
        loss = L + Qsin_W_loss / lambda_w + Qsin_A_loss / lambda_a
        return (loss, outputs) if return_outputs else loss

In [ ]:
args = TrainingArguments(
    "qsin_train_tmp",
    evaluation_strategy = "steps",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_steps=5,
    num_train_epochs=1,
    weight_decay=0.01,
    metric_for_best_model=metric_name,
)

trainer_QSin = QSinTrainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer_QSin.train()